In [21]:
#调用pandas/numpy工具包
import pandas as pd
import numpy as np

In [22]:
#创建数据表
data=[['I1','I2','I5'],['I1','I2'],['I2','I4'],['I1','I2','I4'],['I1','I3'],['I1','I2','I3','I5'],['I1','I2','I3'],['I2','I5'],['I2','I3','I4'],['I3','I4']]

In [23]:
#创建C1函数
def createC1(dataSet):
    #创建C1函数，得到一项集
    C1=[]
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset,C1))

In [24]:
#扫描Data函数，对于每一个项集，扫描data数据，得到每一项的支持度
def scanD(D,CK,minSupport):
    ssCnt = {}
    for tid in D:
        for can in CK:
            if can.issubset(tid):
                if not can in ssCnt:ssCnt[can]=1
                else:ssCnt[can]+=1
    numItems = float(len(D))
    retList = []
    supportData={}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support>=minSupport:
            retList.insert(0,key)
        supportData[key]=support
    return retList,supportData

In [25]:
#频繁项集两两组合
def aprioriGen(Lk,k):
    retList=[]
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1=list(Lk[i])[:k-2];L2=list(Lk[j])[:k-2]
            L1.sort();L2.sort()
            if L1==L2:
                retList.append(Lk[i]|Lk[j])
    return retList

In [26]:

def apriori(dataSet,minSupport=0.2):
    C1=createC1(dataSet)
    D=list(map(set,dataSet))
    L1,supportData =scanD(D,C1,minSupport)
    L=[L1]
    k=2
    while(len(L[k-2])>0):
        CK = aprioriGen(L[k-2],k)
        Lk,supK = scanD(D,CK,minSupport)
        supportData.update(supK)
        L.append(Lk)
        k+=1
    return L,supportData

In [27]:
#规则计算的主函数
def generateRules(L,supportData,minConf=0.8):
    bigRuleList = []
    for i in range(1,len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if(i>1):
                rulesFromConseq(freqSet,H1,supportData,bigRuleList,minConf)
            else:
                calcConf(freqSet,H1,supportData,bigRuleList,minConf)
    return bigRuleList

In [28]:
def calcConf(freqSet,H,supportData,brl,minConf=0.8):
    prunedH=[]
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf>=minConf:
            print (freqSet-conseq,'--->',conseq,'conf:',conf)
            brl.append((freqSet-conseq,conseq,conf))
            prunedH.append(conseq)
    return prunedH

In [29]:
def rulesFromConseq(freqSet,H,supportData,brl,minConf=0.8):
    m = len(H[0])
    if (len(freqSet)>(m+1)):
        Hmp1 = aprioriGen(H,m+1)
        Hmp1 = calcConf(freqSet,Hmp1,supportData,brl,minConf)
        if(len(Hmp1)>1):
            rulesFromConseq(freqSet,Hmp1,supportData,brl,minConf)

In [30]:
if __name__=='__main__':
    dataSet=data
    L,supportData=apriori(dataSet)
    rules = generateRules(L,supportData,minConf=0.8)

frozenset({'I1'}) ---> frozenset({'I2'}) conf: 0.8333333333333334
frozenset({'I5'}) ---> frozenset({'I2'}) conf: 1.0
